# Thống kê dữ liệu ảnh plants sau tiền xử lý
- Metadata: `crawler/data/plants.csv`
- Thư mục ảnh sau tiền xử lý và đã split: `image-classifier/data/{train,val,test}`
- Notebook này tổng hợp số liệu cho từng tập train/val/test, so sánh phân bố lớp và spot-check ảnh trước khi training.


In [ ]:
from pathlib import Path
import random
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from PIL import Image

plt.style.use('seaborn-v0_8')
sns.set_palette('crest')

def find_path(relative_path):
    rel = Path(relative_path)
    cwd = Path.cwd().resolve()
    bases = [cwd, *cwd.parents]
    candidates = []
    for base in bases:
        candidates.append(base / rel)
    for base in bases:
        candidates.append(base / 'medicinal-plant' / rel)
    seen = []
    for cand in candidates:
        if cand in seen:
            continue
        seen.append(cand)
        if cand.exists():
            return cand
    tried = "\n".join(str(p) for p in seen)
    raise FileNotFoundError(f"{relative_path} not found. Tried:\n{tried}")

metadata_path = find_path('crawler/data/plants.csv')
data_root = find_path('image-classifier/data')
split_names = ['train', 'val', 'test']
split_dirs = {split: data_root / split for split in split_names}

project_root = metadata_path.parents[2]
artifacts_dir = project_root / 'notebooks' / 'artifacts' / 'preprocessed'
artifacts_dir.mkdir(parents=True, exist_ok=True)

print('Artifacts dir:', artifacts_dir)
print('Working directory:', Path.cwd())
print('Metadata path:', metadata_path)
print('Data root:', data_root)
for split, folder in split_dirs.items():
    if not folder.exists():
        raise FileNotFoundError(f"Missing folder for split {split}: {folder}")
    num_classes = sum(1 for p in folder.iterdir() if p.is_dir())
    print(f"{split}: {num_classes} lớp ({folder})")

metadata = pd.read_csv(metadata_path)
metadata['ID'] = metadata['ID'].astype(str)
id_to_name = dict(zip(metadata['ID'], metadata['Plant latin name']))
metadata.head()


## Thống kê ban đầu
- Số ảnh, số lớp và ảnh trung bình trên lớp của từng tập train/val/test.
- Tổng hợp toàn bộ ảnh (train+val+test) để xem phân bố lớp.


In [ ]:
def scan_class_counts(image_root: Path):
    counts = {}
    for cls_dir in sorted(image_root.iterdir()):
        if cls_dir.is_dir():
            n_images = sum(1 for p in cls_dir.iterdir() if p.is_file())
            counts[cls_dir.name] = n_images
    return counts

split_counts = {split: scan_class_counts(folder) for split, folder in split_dirs.items()}

split_summary = []
for split, counts in split_counts.items():
    total = sum(counts.values())
    num_classes = len(counts)
    avg = total / num_classes if num_classes else 0
    split_summary.append(
        {
            'tập': split,
            'tổng ảnh': total,
            'số lớp': num_classes,
            'ảnh trung bình mỗi lớp': avg,
        }
    )

split_summary_df = pd.DataFrame(split_summary)
display(split_summary_df)

counts_frames = []
for split, counts in split_counts.items():
    df = (
        pd.Series(counts, name='số ảnh')
        .reset_index()
        .rename(columns={'index': 'mã lớp'})
        .assign(tên_loài=lambda df: df['mã lớp'].map(id_to_name), tập=split)
    )
    counts_frames.append(df)

counts_by_split = (
    pd.concat(counts_frames, ignore_index=True)
    if counts_frames
    else pd.DataFrame(columns=['mã lớp', 'số ảnh', 'tên_loài', 'tập'])
)

counts_df = (
    counts_by_split.groupby('mã lớp')['số ảnh']
    .sum()
    .reset_index()
    .assign(tên_loài=lambda df: df['mã lớp'].map(id_to_name))
)

total_images = counts_df['số ảnh'].sum()
num_classes = len(counts_df)
avg_per_class = total_images / num_classes if num_classes else 0

overall_summary = pd.DataFrame(
    {
        'chỉ số': ['Tổng số ảnh', 'Số lớp', 'Số ảnh trung bình mỗi lớp'],
        'giá trị': [total_images, num_classes, avg_per_class],
    }
)

display(overall_summary)
counts_df.head()


## Phân bố số lượng ảnh theo lớp
- Histogram số ảnh mỗi lớp cho từng tập train/val/test.
- Bar chart các lớp có nhiều ảnh nhất trên toàn bộ dữ liệu (gộp train+val+test).


In [ ]:
if counts_by_split.empty:
    print("Không có dữ liệu class để vẽ histogram.")
else:
    fig_split, axes = plt.subplots(1, len(split_dirs), figsize=(16, 4), sharey=True)
    if len(split_dirs) == 1:
        axes = [axes]
    for ax, split in zip(axes, split_dirs.keys()):
        df = counts_by_split[counts_by_split['tập'] == split]
        sns.histplot(df['số ảnh'], bins=30, ax=ax)
        ax.set_title(f'Histogram số ảnh mỗi lớp - {split}')
        ax.set_xlabel('Số ảnh')
        ax.set_ylabel('Số lớp')
    fig_split.tight_layout()
    fig_split.savefig(artifacts_dir / 'pre_split_class_count_histogram.png', dpi=300)
    plt.show()

    top_k = counts_df.sort_values('số ảnh', ascending=False).head(20)
    fig_top, ax_top = plt.subplots(figsize=(8, 6))
    sns.barplot(data=top_k, y='mã lớp', x='số ảnh', ax=ax_top)
    ax_top.set_title('Top 20 lớp có nhiều ảnh nhất (train+val+test)')
    ax_top.set_xlabel('Số ảnh')
    ax_top.set_ylabel('Mã lớp')
    fig_top.tight_layout()
    fig_top.savefig(artifacts_dir / 'pre_top20_classes.png', dpi=300, bbox_inches='tight')
    plt.show()

bottom_k = counts_df.sort_values('số ảnh', ascending=True).head(20)
bottom_k[['mã lớp', 'tên_loài', 'số ảnh']]


### Số lớp có ít ảnh (train+val+test)
- Kiểm tra số lớp nằm dưới ngưỡng ảnh (mặc định 500).


In [ ]:
if counts_df.empty:
    print("Không có dữ liệu class để đếm.")
else:
    threshold = 500
    num_classes = len(counts_df)
    num_below = (counts_df['số ảnh'] < threshold).sum()
    pct_below = num_below / num_classes * 100 if num_classes else 0
    print(f'{num_below}/{num_classes} lớp có ít hơn {threshold} ảnh ({pct_below:.2f}%)')
    counts_df[counts_df['số ảnh'] < threshold]


### Top 20 lớp có ít ảnh nhất (train+val+test)
- Bar chart để xem các lớp thiếu dữ liệu.


In [ ]:
bottom_20 = counts_df.sort_values('số ảnh', ascending=True).head(20).copy()
bottom_20['nhãn'] = bottom_20['tên_loài'].fillna(bottom_20['mã lớp']).str.slice(0, 60)

fig, ax = plt.subplots(figsize=(8, 6))
sns.barplot(data=bottom_20, y='nhãn', x='số ảnh', color='salmon', ax=ax)
ax.set_title('Top 20 lớp có ít ảnh nhất')
ax.set_xlabel('Số ảnh')
ax.set_ylabel('Lớp / Loài')
plt.tight_layout()
fig.savefig(artifacts_dir / 'pre_bottom_20_classes.png', dpi=300, bbox_inches='tight')
plt.show()


### Ảnh trùng giữa các lớp (gộp train/val/test)
- Đếm ảnh trùng tên xuất hiện ở nhiều lớp và top lớp bị ảnh hưởng.


In [ ]:
if counts_df.empty:
    print("Không có dữ liệu class để kiểm tra trùng ảnh.")
else:
    from collections import defaultdict, Counter

    file_to_classes = defaultdict(set)
    for split, split_root in split_dirs.items():
        for cls_dir in split_root.iterdir():
            if cls_dir.is_dir():
                for p in cls_dir.iterdir():
                    if p.is_file():
                        file_to_classes[p.name].add(cls_dir.name)

    dup_files = {name: classes for name, classes in file_to_classes.items() if len(classes) > 1}
    group_strings = sorted({', '.join(sorted(classes)) for classes in dup_files.values()})
    if group_strings:
        print('Nhóm lớp có ảnh trùng (mỗi dòng là danh sách lớp, phân tách dấu phẩy):')
        for group in group_strings:
            print(group)
    total_dup_files = len(dup_files)
    if total_dup_files == 0:
        print("Không tìm thấy ảnh trùng tên giữa các lớp.")
    else:
        class_dup_counts = Counter()
        for classes in dup_files.values():
            for cls in classes:
                class_dup_counts[cls] += 1

        dup_df = (
            pd.Series(class_dup_counts, name='số ảnh trùng')
            .sort_values(ascending=False)
            .reset_index()
            .rename(columns={'index': 'mã lớp'})
        )
        top_n = min(20, len(dup_df))
        top = dup_df.head(top_n)

        fig, ax = plt.subplots(figsize=(8, 6))
        sns.barplot(data=top, y='mã lớp', x='số ảnh trùng', color='steelblue', ax=ax)
        ax.set_title(f'Top {top_n} lớp có nhiều ảnh trùng với lớp khác (mọi split)')
        ax.set_xlabel('Số ảnh trùng (theo tên file)')
        ax.set_ylabel('Mã lớp')
        fig.tight_layout()
        fig.savefig(artifacts_dir / 'pre_duplicates_per_class.png', dpi=300, bbox_inches='tight')
        plt.show()

        print(f'Tổng số file trùng tên giữa các lớp: {total_dup_files}')


### Heatmap ảnh trùng giữa các lớp (train/val/test)
- Ma trận số lượng file trùng tên giữa các cặp lớp (top lớp bị ảnh hưởng).


In [ ]:
if counts_df.empty:
    print('Không có dữ liệu class để vẽ heatmap trùng ảnh.')
else:
    from collections import defaultdict, Counter
    from itertools import combinations

    file_to_classes = defaultdict(set)
    for split, split_root in split_dirs.items():
        for cls_dir in split_root.iterdir():
            if cls_dir.is_dir():
                for p in cls_dir.iterdir():
                    if p.is_file():
                        file_to_classes[p.name].add(cls_dir.name)

    pair_counts = Counter()
    for classes in file_to_classes.values():
        if len(classes) > 1:
            for a, b in combinations(sorted(classes), 2):
                pair_counts[(a, b)] += 1

    if not pair_counts:
        print('Không có ảnh trùng giữa các lớp.')
    else:
        class_totals = Counter()
        for (a, b), c in pair_counts.items():
            class_totals[a] += c
            class_totals[b] += c
        top_classes = [cls for cls, _ in class_totals.most_common(20)]
        matrix = pd.DataFrame(0, index=top_classes, columns=top_classes, dtype=int)
        for (a, b), c in pair_counts.items():
            if a in matrix.index and b in matrix.columns:
                matrix.loc[a, b] = c
                matrix.loc[b, a] = c
        plt.figure(figsize=(8, 6))
        sns.heatmap(matrix, annot=True, fmt='d', cmap='YlGnBu')
        plt.title('Heatmap ảnh trùng giữa các lớp (top 20)')
        plt.xlabel('Lớp')
        plt.ylabel('Lớp')
        plt.tight_layout()
        plt.savefig(artifacts_dir / 'pre_duplicates_heatmap.png', dpi=300, bbox_inches='tight')
        plt.show()


## Thống kê kích thước ảnh
- Phân bố chiều rộng, chiều cao, cạnh ngắn sau tiền xử lý (gộp train/val/test).
- Kiểm tra số ảnh có chiều < ngưỡng (mặc định 224px).
- Scatter width vs height để xem tỷ lệ khung hình.


In [ ]:
threshold = 224
size_records = []
read_errors = 0

for split, split_root in split_dirs.items():
    for cls_dir in split_root.iterdir():
        if cls_dir.is_dir():
            for p in cls_dir.iterdir():
                if p.is_file():
                    try:
                        with Image.open(p) as img:
                            w, h = img.size
                    except Exception:
                        read_errors += 1
                        continue
                    size_records.append({'class_id': cls_dir.name, 'width': w, 'height': h, 'tập': split})

size_df = pd.DataFrame(size_records)

if size_df.empty:
    print('Không có dữ liệu kích thước ảnh.')
else:
    size_df['min_side'] = size_df[['width', 'height']].min(axis=1)
    summary = size_df[['width', 'height', 'min_side']].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])
    display(summary)

    small = size_df[(size_df['width'] < threshold) | (size_df['height'] < threshold)]
    num_small = len(small)
    pct_small = num_small / len(size_df) * 100 if len(size_df) else 0
    print(f"{num_small}/{len(size_df)} ảnh có chiều < {threshold}px ({pct_small:.2f}%)")
    if read_errors:
        print(f'Lỗi khi đọc ảnh: {read_errors}')

    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    sns.histplot(size_df['width'], bins=40, ax=axes[0])
    axes[0].set_title('Phân bố chiều rộng')
    axes[0].set_xlabel('Width (px)')
    axes[0].set_ylabel('Số ảnh')

    sns.histplot(size_df['height'], bins=40, ax=axes[1])
    axes[1].set_title('Phân bố chiều cao')
    axes[1].set_xlabel('Height (px)')
    axes[1].set_ylabel('Số ảnh')

    sns.histplot(size_df['min_side'], bins=40, ax=axes[2])
    axes[2].set_title('Phân bố cạnh ngắn')
    axes[2].set_xlabel('Min(width, height) (px)')
    axes[2].set_ylabel('Số ảnh')

    plt.tight_layout()
    plt.show()

    sample_df = size_df.sample(min(len(size_df), 20000), random_state=0)
    plt.figure(figsize=(7, 6))
    sns.scatterplot(data=sample_df, x='width', y='height', s=10, alpha=0.5)
    plt.axvline(threshold, color='red', linestyle='--', linewidth=1, label=f'Threshold {threshold}px')
    plt.axhline(threshold, color='red', linestyle='--', linewidth=1)
    plt.title('Scatter kích thước ảnh (width vs height)')
    plt.xlabel('Width (px)')
    plt.ylabel('Height (px)')
    plt.legend()
    plt.tight_layout()
    plt.savefig(artifacts_dir / 'pre_scatter_size.png', dpi=300, bbox_inches='tight')
    plt.show()


### Bar chart số ảnh theo lớp (train+val+test)
- Trục X: tên loài (hoặc mã lớp nếu thiếu tên). Trục Y: số ảnh.


In [ ]:
sorted_counts = counts_df.copy()
sorted_counts['nhãn'] = sorted_counts['tên_loài'].fillna(sorted_counts['mã lớp']).str.slice(0, 60)
sorted_counts = sorted_counts.sort_values('số ảnh', ascending=False)

fig, ax = plt.subplots(figsize=(max(12, len(sorted_counts) * 0.2), 6))
sns.barplot(data=sorted_counts, x='nhãn', y='số ảnh', color='steelblue', ax=ax)
plt.title('Số ảnh mỗi lớp (train+val+test, sắp xếp giảm dần)')
plt.xlabel('Loài / Lớp')
plt.ylabel('Số ảnh')
plt.xticks(rotation=90)
ax.margins(x=0)
fig.tight_layout()
fig.savefig(artifacts_dir / 'pre_counts_per_class.png', dpi=300, bbox_inches='tight')
plt.show()


## Lưới ảnh ngẫu nhiên
- Lấy mẫu nhiều lớp (mặc định từ tập train), hiển thị grid để xem trạng thái trước training.


In [ ]:
def sample_image_paths(image_root: Path, n_samples: int = 16):
    all_paths = []
    for cls_dir in image_root.iterdir():
        if cls_dir.is_dir():
            all_paths.extend([p for p in cls_dir.iterdir() if p.is_file()])
    random.shuffle(all_paths)
    return all_paths[:n_samples]

sample_split = 'train'
sample_root = split_dirs[sample_split]
sample_paths = sample_image_paths(sample_root, n_samples=16)

n_cols = 4
n_rows = int(np.ceil(len(sample_paths) / n_cols))
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 3 * n_rows))
axes = axes.flatten()

for ax, img_path in zip(axes, sample_paths):
    try:
        img = Image.open(img_path).convert('RGB')
        ax.imshow(img)
        cls_id = img_path.parent.name
        ax.set_title(f'{sample_split} - lớp {cls_id}')
        ax.axis('off')
    except Exception as exc:
        ax.axis('off')
        ax.set_title(f'Lỗi: {exc}')

for ax in axes[len(sample_paths):]:
    ax.axis('off')
plt.tight_layout()
plt.show()


## Ảnh sau augmentation (train)
- Lấy ngẫu nhiên vài ảnh gốc ở tập train và áp dụng pipeline augmentation (theo config training) nhiều lần để xem biến đổi.


In [ ]:
import sys
import yaml

sys.path.append(str(project_root / 'image-classifier' / 'src'))
from data import build_transforms, IMAGENET_MEAN, IMAGENET_STD

config_path = find_path('image-classifier/config.yaml')
cfg = yaml.safe_load(config_path.read_text())
img_size = cfg.get('img_size', 224)
use_timm_augment = cfg.get('use_timm_augment', False)

# data_cfg tối thiểu để kích hoạt timm augmentation khi enable
base_data_cfg = {
    'input_size': (3, img_size, img_size),
    'mean': IMAGENET_MEAN,
    'std': IMAGENET_STD,
    'interpolation': 'bicubic',
    'crop_pct': 0.95,
}
train_transform = build_transforms(
    img_size=img_size,
    is_train=True,
    data_cfg=base_data_cfg if use_timm_augment else None,
    use_timm_augment=use_timm_augment,
)

# chuẩn hóa để hiển thị
_display_mean = np.array(base_data_cfg.get('mean', IMAGENET_MEAN))
_display_std = np.array(base_data_cfg.get('std', IMAGENET_STD))

def tensor_to_image(tensor):
    arr = tensor.detach().cpu().numpy()
    arr = np.transpose(arr, (1, 2, 0))
    arr = (arr * _display_std) + _display_mean
    arr = np.clip(arr, 0, 1)
    return arr

aug_per_image = 4
sample_paths = sample_image_paths(split_dirs['train'], n_samples=4)
fig, axes = plt.subplots(len(sample_paths), aug_per_image + 1, figsize=(3.2 * (aug_per_image + 1), 3 * len(sample_paths)))

for row, img_path in enumerate(sample_paths):
    orig = Image.open(img_path).convert('RGB').resize((img_size, img_size))
    cls_id = img_path.parent.name
    axes[row, 0].imshow(orig)
    axes[row, 0].set_title(f'gốc - lớp {cls_id}')
    axes[row, 0].axis('off')

    for i in range(aug_per_image):
        augmented = train_transform(orig)
        if isinstance(augmented, tuple):
            augmented = augmented[0]
        arr = tensor_to_image(augmented)
        axes[row, i + 1].imshow(arr)
        axes[row, i + 1].set_title(f'aug {i + 1}')
        axes[row, i + 1].axis('off')

fig.tight_layout()
fig.savefig(artifacts_dir / 'pre_train_augmentations.png', dpi=300, bbox_inches='tight')
plt.show()
